# AZML III Pipeline - hyper parametre et Auto ml
Laurent Cetinsoy


## AutoML

Le machine learning, c'est bien. Le terme buzz du moment c'est AutoML : automatiser la recherche du meilleur modèle ou des meilleurs hyper paramètres pour un dataset donné. Plein d'entreprises proposent des outils pour le faire et microsoft ne déroge pas à la règle : le sdk permet d'en faire.  

Reférence utile :

- https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
- https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml

Les concepts / classes utiles sont : AutoMLConfig

Charger le Workspace comme d'hab

from azureml.core import Workspace
ws = Workspace.create(name='azml1_lk',
                      subscription_id='50f55159-0c1b-4154-9452-e079e6b5e663',
                      resource_group='AZML',
                      location = 'northeurope')

In [1]:
from azureml.core import Workspace
ws = Workspace.from_config("config.json")

### Regression

On va entraîner un modèle de régression sur le dataset house. Charger le dataset house.csv avec pandas

In [2]:
import pandas as pd
df = pd.read_csv("house.csv")
df.head()

,size,nb_rooms,garden,orientation,price
0,197.330478,1,0,Sud,5.156551e+05
1,185.361036,2,0,Sud,9.341416e+05
2,195.113286,3,0,Est,6.894269e+06
3,139.920144,3,1,Ouest,4.307298e+06
4,167.917466,1,0,Nord,1.267890e+06


In [4]:
# On prend un autre dataset car résultats pas très cohérents
from sklearn.datasets import load_boston
import pandas as pd
df = pd.read_csv(load_boston().filename, skiprows=0, header=1)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Faire un train test split

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.2, random_state=64)
X_train.shape, X_test.shape

((404, 14), (102, 14))

Créer un objet de la classe AutoMLConfig. Configurer l'experience pour que ça ne dure pas trop longtemps  avec notamment les paramètre experiment_timeout_minute et n_cross_validations

In [6]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "iteration_timeout_minutes": 2,
    "experiment_timeout_minutes": 20,
    #"n_cross_validations": 5,
    "featurization": 'auto'
}

automl_config = AutoMLConfig(task='regression',
                             primary_metric='r2_score',
                             training_data = X_train,
                             validation_data = X_test,
                             label_column_name="MEDV",
                             **automl_settings)

Créer une experience "myautoml_classification"

In [7]:
from azureml.core.experiment import Experiment
myautoml_reg = Experiment(ws, "boston_house_regression")

Lancer votre experience avec le submit sur l'objet de config

In [8]:
local_run = myautoml_reg.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_04299c0b-70b6-412d-9870-31ad10dd4f59

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and 

In [9]:
best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: boston_house_regression,
Id: AutoML_04299c0b-70b6-412d-9870-31ad10dd4f59_29,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, force_text_dnn=None,
        is_cross_validation=None, is_onnx_compatible=None, logger=None,
        obser...      subsample=0.6, tree_method='auto', validate_parameters=1,
         verbose=-10, verbosity=0))]),
          stddev=None)


Si ça fail, ce n'est pas très grave. Essayez de débug mais ne perdez pas trop de temps. Automl a besoin de bcp de librairies et le faire marcher en local est galère. Passer plutôt en remote ou lancer un notebook sur votre workspace azure

## Option en remote

Pour cela on va devoir utiliser une ressource en remote. 
Récupérer une référence au AmlCompute du notebook précédent si vous l'avez fait.
Sinon en créér un grâce à la méthode provisioning_configuration de la classe AmlCompute

In [10]:
from azureml.core.compute import AmlCompute
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

Créer une instance de la classe ComputeTarget

In [11]:
from azureml.core.compute import ComputeTarget
cpu_cluster = ComputeTarget.create(ws, "elkalkul", compute_config)

In [30]:
#Création d'un CT plus puissante pour aller plus vite
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS4_V2', max_nodes=8)
cpu_cluster = ComputeTarget.create(ws, "elkalkul-big", compute_config)

### Upload du dataset

en remote on est obligé d'upload notre dataset sur un DataStore et récupérer une référence via un `Dataset`. En effet on ne peut pas envoyer tel quel un df pandas en mémoire vers la compute target distante.

Uploader et récupérer une référence du fichier houses_automl.csv

In [12]:
ds = ws.get_default_datastore()
ds.upload_files(['./house.csv'])

Uploading an estimated of 1 files
Target already exists. Skipping upload for house.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [13]:
#on réécrit un csv avec la première ligne en moins pour pouvoir ensuite le charger sur le datastore
df.to_csv('boston_house.csv', index=False)

In [14]:
ds = ws.get_default_datastore()
ds.upload_files(['./boston_house.csv'])

Uploading an estimated of 1 files
Target already exists. Skipping upload for boston_house.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [15]:
from azureml.core import Dataset
source = [(ds, 'boston_house.csv')]
dataset = Dataset.Tabular.from_delimited_files(path=source)

In [16]:
data_test, data_train = dataset.random_split(0.2, seed=64)

Créer une AutoMLConfig qui utilise le dataset uploadé et la compute target à distance

In [17]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "iteration_timeout_minutes": 2,
    "experiment_timeout_hours": 0.3,
    "featurization": 'auto',
    "max_concurrent_iterations": 4#,
    #"n_cross_validations": 5
}

automl_config = AutoMLConfig(task='regression',
                             primary_metric='r2_score',
                             compute_target=cpu_cluster,
                             training_data = data_train,
                             validation_data = data_test,
                             label_column_name="MEDV",
                             **automl_settings)

Créer une experience "remoteautoml". Submit l'experience avec l'objet automl config que vous venez de créer

In [18]:
remoteautoml = Experiment(ws, "boston_house_regression_remote")
remote_run = remoteautoml.submit(automl_config, show_output=True)

Running on remote or ADB.
Running on remote compute: elkalkul
Parent Run ID: AutoML_4534478e-1ddc-42ba-9cb6-0cce7d7ce39a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

**************

```python
best_run, fitted_model = remote_run.get_output()
```
__/!\ /!\ /!\__ Problème avec la sérialisation du XGboost, pour simplifier on peut éventuellement mettre XGBoost et LightGBM dans le paramètre blocked_models pour avoir que des moèles de sklearn de sorte qu'il y ait pas ce souci. Sinon il faut fouiller pour le résoudre (la difficulté étant que c'est autoML qui s'occupe d'enregistrer/sérialiser les modèles donc il s'agit de trouver comment avoir la main là-dessus)

Vérifier que cela a bien marché et récupérer le meilleur modèle

## Hyper drive 

Kezako ? Module d'azure machine learning qui permet de faire de la recherche d'hyper parametres.

Reférences utiles : 
- https://docs.microsoft.com/fr-fr/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py
- https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters


Les classes concepts intéressant sont HyperParameterSampling, HyperDriveRun

Quelle différence entre hyperdrive et automl ? C'est pas la même chose ? Pourquoi deux choses différentes ? 

automl teste plusieurs modèles, hyperdrive teste plusieurs hyperparamètres d'un même modèle

A quoi sert la classe HyperParameterSampling ? 


Permet de spécifier comment on "chercher" des hyperparametres (cf https://docs.microsoft.com/fr-fr/python/api/azureml-train-core/azureml.train.hyperdrive.hyperparametersampling?view=azure-ml-py)

Quelles sont les principales classes filles ? 

"Cette classe encapsule l’espace hyperparamétrique, la méthode d’échantillonnage et les propriétés supplémentaires pour les classes d’échantillonnage dérivées : BayesianParameterSampling , GridParameterSampling et RandomParameterSampling ."

Quelles semblent-être les méthodes les plus importantes ? 

A quoi sert la classe HyperDriveRun ? 

Classe fille de la classe run, c'est une classe particulière qui permet de récupérer les résultats de la recherche d'hyperparamètres.

On va faire la recherche d'hyperparamètre sur notre modèle de régression de tout à l'heure (house).
Créer un dictionnaire avec les noms des paramètres que vous voulez recherche (par exemple le learning rate ou le taux de régularisation), cf https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#define-search-space

Les faire commencer par un --. Par exemple --n_epochs et pas n_epochs. Cela servira pour passer les arguments à notre script d'entraînement

In [51]:
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
param_dict = {
    "--loss": choice('ls', 'lad', 'huber'),
    "--learning_rate": uniform(0.05, 0.1),
    "--n_estimators": choice(50, 100, 250, 500, 1000),
    "--criterion": choice('friedman_mse', 'mse', 'mae'),
    "--max_depth": choice(3, 5, 7)
}

Instancier un objet de la classe RandomParameterSampling 

In [52]:
from azureml.train.hyperdrive.sampling import RandomParameterSampling
param_sampling = RandomParameterSampling(param_dict)

Spécifier la métrique que vous voulez optimiser

On va maximiser le R2 qu'on déclarera dans les paramètres de HyperDriveConfig

Choisir un critère d'arrêt de la recherche d'hyper paramètre 

In [53]:
from azureml.train.hyperdrive import MedianStoppingPolicy
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

Editer le fichier d'entraînement house_training.py pour qu'il soit compatible avec l'entraînement. Il faut utiliser le module argpass pour récupérer les paramètres dont vous voulez faire la recherche. Cf https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

! Editer le fichier !

Créer un estimateur SkLearn

In [54]:
from azureml.train.sklearn import SKLearn
estimator = SKLearn(source_directory=".", 
                    compute_target=cpu_cluster,
                    entry_script='train_house.py'
                   )

Créer une configuration HyperdriveConfiguration

In [55]:
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal

hyperdrive_config = HyperDriveConfig(estimator=estimator,
                          hyperparameter_sampling=param_sampling, 
                          policy=early_termination_policy,
                          primary_metric_name="r2_score", 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs=100,
                          max_concurrent_runs=4)

Soumettre l'experience localement

In [56]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "hyper_param_tuning")
hyperdrive_run = experiment.submit(hyperdrive_config)

Et récupérer le meilleur set d'hyperparamètres

In [50]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(parameter_values)
print(best_run_metrics)

['--learning_rate', '0.06967842644937153', '--loss', 'ls', '--n_estimators', '500']
{'r2_score': 0.8950570510468429, 'Loss function': 'ls', 'Learning Rate': 0.06967842644937153, 'Number of Estimators': 500, 'MSE': 10.228252775530551}
